# Modèle pour la détermination du type de véhicule

### 1. Préparation des données

In [ ]:
# Directories

ORIGINAL_DATA_DIR = './original_data'
PREPROCESSED_DATA_DIR = './preprocessed_data'
TRAIN_OUTPUT_FOLDER = PREPROCESSED_DATA_DIR+'/train'
VALID_OUTPUT_FOLDER = PREPROCESSED_DATA_DIR+'/valid'
TESTS_OUTPUT_FOLDER = PREPROCESSED_DATA_DIR+'/test'
MODEL1_IMAGE_SAMPLE_DIR = './model1_image_samples'
MODEL_DIR = './resulting_models'


Crop provisoire des images

In [49]:
import os
from PIL import Image

def crop_images (dossier_crops, dossier_coordonnees, dossier_images) :
    # Obtenir les listes triées des fichiers dans les dossiers
    fichiers_images = sorted(os.listdir(dossier_images))
    fichiers_coordonnees = sorted(os.listdir(dossier_coordonnees))

    # Vérifier si les deux dossiers ont le même nombre de fichiers
    if len(fichiers_images) != len(fichiers_coordonnees):
        print("Le nombre de fichiers dans les deux dossiers ne correspond pas.")
        exit()

    # Parcours des images et fichiers de coordonnées
    for index, image_file in enumerate(fichiers_images):
        if image_file.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(dossier_images, image_file)
            coord_file = os.path.join(dossier_coordonnees, fichiers_coordonnees[index])
            
            # Charger l'image
            with Image.open(image_path) as img:
                img_width, img_height = img.size  # Taille de l'image
                # Lire les coordonnées
                with open(coord_file, "r") as file:
                    lines = file.readlines()
                    for i, line in enumerate(lines):
                        data = line.strip().split()
                        
                        # Vérifier la validité des données
                        if len(data) < 9:
                            print(f"Coordonnées invalides dans {coord_file}, ligne {i + 1}")
                            continue

                        img_type = data[-2]

                        # Calcul des coordonnées absolues du rectangle (x_min, y_min, x_max, y_max)
                        x_min = int(round(float(data[2])))
                        y_min = int(round(float(data[1])))
                        x_max = int(round(float(data[0])))
                        y_max = int(round(float(data[5])))

                        # Découper l'image
                        cropped_img = img.crop((x_min, y_min, x_max, y_max))
                        
                        # Sauvegarder l'image cropée
                        crop_file_name = f"{img_type}_{index + 1}_{i + 1}.jpg"
                        crop_path = os.path.join(dossier_crops, crop_file_name)
                        cropped_img.save(crop_path, "JPEG")


Transformation : Niveau de gris sur tout le dataset

In [50]:
import cv2, os

def change_to_gray (input_folder, output_folder) :
    """
    Transforme une image à l'origine en couleur en niveaux de gris
    """
    for image_name in os.listdir(input_folder) :
        img = cv2.imread(f'{input_folder}/{image_name}')
        grayFrame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        output_path = os.path.join(output_folder, image_name)
        cv2.imwrite(output_path, grayFrame)


Transformation : ajout de bruit gaussien sur les images pour simuler une caméra de mauvaise qualité

In [51]:
import numpy as np
from PIL import Image
import random

def add_gaussian_noise(img, mean=0, std=25):
    """
    Ajoute un bruit gaussien à une image.
    """
    # Convertir l'image en tableau numpy
    img_array = np.array(img, dtype=np.float32)

    # Générer du bruit gaussien
    std = 10
    noise = np.random.normal(mean, std, img_array.shape)
    noisy_img = img_array + noise

    # Limiter les valeurs entre 0 et 255
    noisy_img = np.clip(noisy_img, 0, 255).astype(np.uint8)
    return noisy_img

def process_images_with_effects(source_dir, target_dir, apply_blur=True, apply_noise=True):
    """
    Applique flou et bruit aux images redimensionnées.
    """
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    for filename in os.listdir(source_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            file_path = os.path.join(source_dir, filename)
            save_path = os.path.join(target_dir, filename)

            # Charger l'image
            with Image.open(file_path) as img:
                # Convertir en BGR pour OpenCV
                img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

                # Appliquer bruit gaussien
                if apply_noise and random.random() > 0.3:  # 50% de chances d'ajouter du bruit
                    img_cv = add_gaussian_noise(img_cv, mean=0, std=random.randint(15, 40))

                # Convertir en RGB et sauvegarder
                final_img = Image.fromarray(cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB))
                final_img.save(save_path)

Transformation : Redimensionnement pour que les images soient environ de la taille de la médiane des tailles des exemples

In [52]:
from PIL import Image
import os
import numpy as np
import random

def calculate_median_size(image_paths):
    """Calcul la médiane des tailles (largeur, hauteur) des images."""
    dimensions = []
    for path in image_paths:
        with Image.open(path) as img:
            dimensions.append(img.size)  # (width, height)
    dimensions = np.array(dimensions)
    median_width = int(np.median(dimensions[:, 0]))
    median_height = int(np.median(dimensions[:, 1]))
    return median_width, median_height

def add_random_variation(size, variation_percent=0.2):
    """
    Ajoute une variation aléatoire à une taille donnée.
    """
    factor = 1 + random.uniform(-variation_percent, variation_percent)
    return int(size * factor)

def resize_image_with_aspect_ratio(img, target_width, target_height):
    """
    Redimensionne une image en conservant le ratio d'aspect, en s'adaptant à la taille cible.
    """
    original_width, original_height = img.size
    aspect_ratio = original_width / original_height

    # Ajuster la largeur et la hauteur pour conserver le ratio
    if target_width / target_height > aspect_ratio:
        # Ajuste par la hauteur
        new_height = target_height
        new_width = int(new_height * aspect_ratio)
    else:
        # Ajuste par la largeur
        new_width = target_width
        new_height = int(new_width / aspect_ratio)

    try :
        return img.resize((new_width, new_height), Image.ANTIALIAS)
    except :
        return img.resize((new_width, new_height), Image.Resampling.LANCZOS)

def resize_images(source_dir, target_dir, median_size, variation_percent=0.2):
    """
    Redimensionne les images en conservant leur ratio d'aspect, avec des tailles légèrement différentes.
    """
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    for filename in os.listdir(source_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            file_path = os.path.join(source_dir, filename)
            with Image.open(file_path) as img:
                # Ajouter une variation aléatoire aux dimensions médianes
                target_width = add_random_variation(median_size[0], variation_percent)
                target_height = add_random_variation(median_size[1], variation_percent)

                # Redimensionner en conservant le ratio d'aspect
                resized_img = resize_image_with_aspect_ratio(img, target_width, target_height)

                # Sauvegarder l'image redimensionnée
                save_path = os.path.join(target_dir, filename)
                resized_img.save(save_path)


Transformation : UpSampling en retournant les images aléatoirement
Tous les exemples uniques de rotation/flip d'image :

| Rotation (k) | Flip Horizontal | Flip Vertical |
|--------------|-----------------|---------------|
| 0            | False           | False         |
| 0            | True            | False         |
| 0            | False           | True          |
| 1            | False           | False         |
| 1            | True            | False         |
| 2            | False           | False         |
| 2            | True            | False         |
| 3            | False           | False         |
| 3            | True            | False         |

In [68]:
import tensorflow as tf
import numpy as np

unique_combinations = [
    # (0, False, False), ==> Image de base : (nb de flip à 90°, mirroir horizontal, mirroir veritcal)
    (0, True, False),
    (0, False, True),
    (1, False, False),
    (1, True, False),
    (2, False, False),
    (2, True, False),
    (3, False, False),
    (3, True, False)
]

def apply_transformations(img, rotation_count, flip_horizontal, flip_vertical):
    """
    Applique une transformation à l'image selon le nombre de rotations et les flips.
    """
    img = tf.convert_to_tensor(img, dtype=tf.float32)

    img = tf.image.rot90(img, k=rotation_count)

    # Appliquer les flips
    if flip_horizontal:
        img = tf.image.flip_left_right(img)
    if flip_vertical:
        img = tf.image.flip_up_down(img)

    img_rotated = img.numpy().astype(np.uint8)
    return img_rotated

def rotate_images (input_folder, output_folder) :
    for image_name in os.listdir(input_folder) :
        img = cv2.imread(f'{input_folder}/{image_name}')

        # On choisit deux transformations aléatoires
        random_choice1 = random.choice(unique_combinations)
        random_choice2 = random.choice(unique_combinations)

        # On vérifie que les choix sont différents
        while random_choice1 == random_choice2:
            random_choice2 = random.choice(unique_combinations)

        rotation_count1, flip_horizontal1, flip_vertical1 = random_choice1
        rotation_count2, flip_horizontal2, flip_vertical2 = random_choice2

        rotations_list = [apply_transformations(img, rotation_count1, flip_horizontal1, flip_vertical1), 
                        apply_transformations(img, rotation_count2, flip_horizontal2, flip_vertical2)]
        

        img_name = os.path.splitext(image_name)[0]
        category = img_name.split("_")[0]
        output_path = os.path.join(output_folder + f'/{category}/', f"{img_name}.png")
        cv2.imwrite(output_path, img)

        for i, rotated_image in enumerate(rotations_list) :
            output_image_name = f"{img_name}_{i+1}.png"
            os.makedirs(f'{output_folder}/{category}', exist_ok=True)
            output_path = os.path.join(output_folder + f'/{category}/', output_image_name)
            cv2.imwrite(output_path, rotated_image)

Version finale de la préparation des données

In [ ]:
import shutil
import os
from PIL import Image

Image.MAX_IMAGE_PIXELS = None  # Supprime complètement la limite

def prepare_data (mode, output_dir) :
    print(f'Beginning of the preparation process for {mode} data.\n')
    tmp_output_dir = output_dir+'/tmp'

    # Chemins des dossiers
    dossier_images = f"./{ORIGINAL_DATA_DIR}/{mode}/images"  # Chemin vers le dossier contenant les images
    dossier_coordonnees = f"./{ORIGINAL_DATA_DIR}/{mode}/labelTxt"  # Chemin vers le dossier contenant les fichiers de coordonnées
    
    # Créer le dossier de sortie s'il n'existe pas
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(tmp_output_dir, exist_ok=True)
    crop_images (tmp_output_dir, dossier_coordonnees, dossier_images)
    ### la partie au dessus sert à croper provisoirement. Elle sera à enlever plus tard.

    change_to_gray(tmp_output_dir, tmp_output_dir)
    print(f'End of the process Color to Gray')

    # Appliquer flou et bruit
    process_images_with_effects(tmp_output_dir, tmp_output_dir, apply_blur=True, apply_noise=True)
    print(f'End of the process Add Effects on Images')

    # Images avec tailles définies
    defined_images = [os.path.join(MODEL1_IMAGE_SAMPLE_DIR, f) for f in os.listdir(MODEL1_IMAGE_SAMPLE_DIR)
                    if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]

    # Calcul de la taille médiane
    median_size = calculate_median_size(defined_images)

    # Redimensionnement des images avec une variation aléatoire de ±20 %
    resize_images(tmp_output_dir, tmp_output_dir, median_size, variation_percent=0.2)
    print(f'End of the process Resize Images')
    # shutil.rmtree('./noise_images')

    # Génération d'images tournées dans des sens différents
    rotate_images (tmp_output_dir, output_dir)
    print(f'End of the process Random Turn Images')
    shutil.rmtree(tmp_output_dir)

    print(f'\nEnd of the Preparation process')

In [ ]:
prepare_data ('train', f'{TRAIN_OUTPUT_FOLDER}')
prepare_data ('test', f'{TESTS_OUTPUT_FOLDER}')
prepare_data ('validation', f'{VALID_OUTPUT_FOLDER}')

Beginning of the preparation process for train data.

End of the process Color to Gray
End of the process Add Effects on Images
End of the process Resize Images
End of the process Random Turn Images

End of the Preparation process
Beginning of the preparation process for test data.

End of the process Color to Gray
End of the process Add Effects on Images
End of the process Resize Images
End of the process Random Turn Images

End of the Preparation process
Beginning of the preparation process for validation data.

End of the process Color to Gray
End of the process Add Effects on Images
End of the process Resize Images
End of the process Random Turn Images

End of the Preparation process


Importation des datasets

In [72]:
import tensorflow as tf

# Images avec tailles définies
defined_images = [os.path.join(MODEL1_IMAGE_SAMPLE_DIR, f) for f in os.listdir(MODEL1_IMAGE_SAMPLE_DIR)
                if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]

img_width,img_height = calculate_median_size(defined_images)
batch_size = 32

train_dataset = tf.keras.utils.image_dataset_from_directory(
    TRAIN_OUTPUT_FOLDER,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='int',
    color_mode='grayscale'
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    VALID_OUTPUT_FOLDER,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='int',
    color_mode='grayscale'
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    TESTS_OUTPUT_FOLDER,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='int',
    color_mode='grayscale'
)

Found 2292 files belonging to 2 classes.
Found 666 files belonging to 2 classes.
Found 327 files belonging to 2 classes.


Création du modèle

In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint_cb = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True) # Keep best model instead of last
early_stopping_cb = EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True) # Prevent overfitting

model = tf.keras.Sequential([
    # Entrée
    layers.Input(shape=(img_width, img_height, 1)),

    # Bloc convolutionnel 1
    layers.Conv2D(filters=32, kernel_size=(5, 5), activation='relu'),
    layers.BatchNormalization(),  # Ajout d'une normalisation
    layers.MaxPooling2D((2, 2)),

    # Bloc convolutionnel 2
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    # Bloc convolutionnel 3
    layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    layers.BatchNormalization(),

    # Bloc convolutionnel 4 (ajouté pour extraire davantage de caractéristiques complexes)
    layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu'),
    layers.BatchNormalization(),

    # Bloc convolutionnel 5 (couche supplémentaire avec une taille de noyau différente)
    layers.Conv2D(filters=512, kernel_size=(2, 2), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),


    # Aplatissement des caractéristiques
    layers.Flatten(),

    # Couches denses
    layers.Dense(512, activation='relu'),  # Couche dense étendue
    layers.Dropout(0.5),  # Dropout augmenté pour éviter le surapprentissage
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),

    # Sortie binaire
    layers.Dense(1, activation='sigmoid')  # Sortie binaire pour classification
])

# Compilation du modèle
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [81]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_17 (Conv2D)              │ (None, 38, 48, 32)     │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 38, 48, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 19, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 17, 22, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 17, 22, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 8, 11, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 6, 9, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 6, 9, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 4, 7, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 4, 7, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 3, 6, 512)      │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 3, 6, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 1, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 1536)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 512)            │       786,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,502,981 (20.99 MB)

 Trainable params: 1,833,665 (6.99 MB)

 Non-trainable params: 1,984 (7.75 KB)

 Optimizer params: 3,667,332 (13.99 MB)

Training the Model

In [80]:
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=15,
    callbacks=[early_stopping_cb, checkpoint_cb]
)

Epoch 1/15
72/72 ━━━━━━━━━━━━━━━━━━━━ 30s 222ms/step - accuracy: 0.6383 - loss: 1.0819 - val_accuracy: 0.4850 - val_loss: 1.1439
Epoch 2/15
72/72 ━━━━━━━━━━━━━━━━━━━━ 14s 186ms/step - accuracy: 0.7338 - loss: 0.6147 - val_accuracy: 0.6997 - val_loss: 0.5564
Epoch 3/15
72/72 ━━━━━━━━━━━━━━━━━━━━ 13s 176ms/step - accuracy: 0.7454 - loss: 0.5880 - val_accuracy: 0.6652 - val_loss: 0.6202
Epoch 4/15
72/72 ━━━━━━━━━━━━━━━━━━━━ 16s 220ms/step - accuracy: 0.7909 - loss: 0.5056 - val_accuracy: 0.7087 - val_loss: 0.5485
Epoch 5/15
72/72 ━━━━━━━━━━━━━━━━━━━━ 13s 180ms/step - accuracy: 0.8261 - loss: 0.3981 - val_accuracy: 0.7628 - val_loss: 0.5659
Epoch 6/15
72/72 ━━━━━━━━━━━━━━━━━━━━ 13s 174ms/step - accuracy: 0.8524 - loss: 0.3592 - val_accuracy: 0.7838 - val_loss: 0.6346
Epoch 7/15
72/72 ━━━━━━━━━━━━━━━━━━━━ 12s 171ms/step - accuracy: 0.8716 - loss: 0.2914 - val_accuracy: 0.7072 - val_loss: 0.9377
Epoch 8/15
72/72 ━━━━━━━━━━━━━━━━━━━━ 12s 173ms/step - accuracy: 0.8886 - loss: 0.2701 - val_accu

In [85]:
version = "v3.1_many_conv"

os.makedirs(MODEL_DIR, exist_ok=True)
model.save(f'{MODEL_DIR}/model_'+version+'.keras')

Cropped images from the first model

In [86]:
# last_validation = tf.keras.utils.image_dataset_from_directory(
#     "../Model1/images/crop",
#     image_size=(img_height, img_width),
#     batch_size=batch_size,
#     label_mode='int',
#     color_mode='grayscale'
# )

Utilisation des données de test

In [87]:
test_loss, test_accuracy = model.evaluate(test_dataset, verbose=2)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


11/11 - 1s - 50ms/step - accuracy: 0.7034 - loss: 0.6964
Test Loss: 0.6964
Test Accuracy: 0.7034
